<a href="https://colab.research.google.com/github/pedromontero2018Ghb/ReordenamientoGenQAOA/blob/main/breakpennylanebueno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#para qiskit en pennylane se instalan las librerias
!pip install pennylane-qiskit
import pennylane as qml
from pennylane import numpy as np
import math

In [2]:


# -------------------------
# Construcción de Hamiltoniano válido (diagonal con costes)#
# Se construye un Hamiltoniano diagonal donde cada entrada H[i,i] se corresponde
# al coste asociado a la inversión i. Se elige el número de qubits
# -------------------------

def build_cost_hamiltonian(costs, num_qubits):
    dim = 2**num_qubits
    H = np.zeros((dim, dim))
    for i, c in enumerate(costs):
        H[i, i] = float(c)   # matriz diagonal con los costes
    return qml.Hermitian(H, wires=range(num_qubits))

# -------------------------
# Se crea e circuito y se eligen los valores profundidad de QAOA (capas),
# el número de iteraciones del optimador clásico (Adam) y la tasa de aprendizaje.
# -------------------------

def solve_qaoa_step(costs, reps=2, steps=80, stepsize=0.1, seed=1234):
    num_reversals = len(costs)
    if num_reversals <= 1:
        return 0

    num_qubits = max(1, math.ceil(math.log2(num_reversals)))
# El número de qubits crece logarítmicamente con las posibles inversiones.
    dev = qml.device("default.qubit", wires=num_qubits)
# Usa default.qubit de Pennylane
    cost_hamiltonian = build_cost_hamiltonian(costs, num_qubits)
# Penaliza configuraciones con más breakpoints.
    mixer_hamiltonian = qml.Hamiltonian([1.0]*num_qubits,
                                        [qml.PauliX(i) for i in range(num_qubits)])
# El uso de las puertas X exploran el espacio de soluciones.
# A continuación se usa el algoritmo QAOA
# y se devuelve el valor esperado del Hamiltoniano de costes
# (lo que optimizamos).

    @qml.qnode(dev)
    def circuit(params):
        for w in range(num_qubits):
            qml.Hadamard(wires=w) # superposición inicial
        for l in range(reps):
            gamma = params[0][l] # ángulo de coste
            alpha = params[1][l] # ángulo de mezcla
            qml.evolve(cost_hamiltonian, gamma)
            qml.evolve(mixer_hamiltonian, alpha)
        return qml.expval(cost_hamiltonian)

    rng = np.random.default_rng(seed)
    # Inicializa aleatoriamente los parámetros
    params = np.array(rng.uniform(0, np.pi, (2, reps)), requires_grad=True)
    # Usa Adam para ajustarlos minimizando la energía.
    opt = qml.AdamOptimizer(stepsize=stepsize)
    for _ in range(steps):
        params, _ = opt.step_and_cost(circuit, params)
# Igual que el circuito anterior pero una vez entrenado, mide las
# probabilidades de cada estado base.
    @qml.qnode(dev)
    def probs_circuit(params):
        for w in range(num_qubits):
            qml.Hadamard(wires=w)
        for l in range(reps):
            gamma = params[0][l]
            alpha = params[1][l]
            qml.evolve(cost_hamiltonian, gamma)
            qml.evolve(mixer_hamiltonian, alpha)
        return qml.probs(wires=range(num_qubits))

    probs = np.array(probs_circuit(params))
    probs = probs[:num_reversals]
    # Selecciona el estado con mayor probabilidad como la inversión elegida:
    qaoa_idx = int(np.argmax(probs))

    best_classical_idx = int(np.argmin(np.array(costs)))

    return qaoa_idx

# -------------------------
# Utilidades clásicas. Añade centinelas, 0 al principio y n al final
# -------------------------
#  Añade centinelas, 0 al principio y n al final para si es necesario
#para comparar con el caso signado
def int_centinelas(perm):
    n = len(perm)
    return [0] + [int(p) + 1 for p in perm] + [n + 1]
# Calcula los puntos de ruptura
def calcula_breakpoints(perm):
    return sum(1 for i in range(len(perm)-1) if abs(perm[i]-perm[i+1]) != 1)
#Aplica las reversiones
def aplica_reversion(perm, i, j):
    nueva = perm[:]
    nueva[i:j+1] = reversed(nueva[i:j+1])
    return nueva

# -------------------------
# Función principal:Mientras queden puntos de ruptura:
#     Calcula todas las inversiones posibles (i,j).
#     Evalúa cuántos puntos de ruptura tendría cada inversión
#     Llama a solve_qaoa_step(costs) para elegir inversión.
#     Aplica inversión, guarda en el historial, imprime el progreso.
# Finalmente si llega a permutación ordenada imprime un mensaje de éxito
#  y si no, termina

def solve_sorting_by_reversals_pennylane(inicial_permutation):
    actual = int_centinelas(inicial_permutation)
    # se dan valor a algunas de las principales cantidades
    n = len(inicial_permutation)
    reversal_history = []
    step = 1
    max_steps = max(1, n*(n-1)//2)

    print(f"Permutación inicial: {inicial_permutation}")
    print(f"Permutación con centinelas: {actual}")
    print("-"*40)

    while calcula_breakpoints(actual) > 0 and step <= max_steps:
        print(f"Paso {step}:")
        possible_reversals = [(i, j) for i in range(1, n+1) for j in range(i+1, n+1)]
        costs = [calcula_breakpoints(aplica_reversion(actual, i, j))
                 for i, j in possible_reversals]

        print(f"  - Puntos de ruptura actuales: {calcula_breakpoints(actual)}")
        print(f"  - Construyendo Hamiltoniano para {len(possible_reversals)} inversiones...")

        idx = solve_qaoa_step(costs)
        best_rev = possible_reversals[idx]
        new_actual = aplica_reversion(actual, best_rev[0], best_rev[1])

        if calcula_breakpoints(new_actual) >= calcula_breakpoints(actual):
            idx = int(np.argmin(np.array(costs)))
            best_rev = possible_reversals[idx]
            new_actual = aplica_reversion(actual, best_rev[0], best_rev[1])

        actual = new_actual
        reversal_history.append((best_rev[0]-1, best_rev[1]-1))

        print(f"  - Inversión elegida: {(best_rev[0]-1, best_rev[1]-1)}")
        print(f"  - Permutación actual: {[x-1 for x in actual[1:-1]]}")
        print("-"*40)
        step += 1

    if calcula_breakpoints(actual) == 0:
        print("¡Permutación ordenada con éxito!")
    else:
        print("No se alcanzó el orden completo dentro del tope de pasos.")

    print(f"Permutación final: {[x-1 for x in actual[1:-1]]}")
    print(f"Secuencia de inversiones (índices base 0): {reversal_history}")
    print(f"Número total de inversiones: {len(reversal_history)}")

# -------------------------
# Ejecución de ejemplo
# -------------------------

if __name__ == "__main__":
    inicial_permutation = [0, 3, 1, 2]
    solve_sorting_by_reversals_pennylane(inicial_permutation)

    print("\n" + "="*50 + "\n")
    inicial_permutation_2 = [7, 6, 5, 4, 0, 3, 1, 2]
    solve_sorting_by_reversals_pennylane(inicial_permutation_2)


Permutación inicial: [0, 3, 1, 2]
Permutación con centinelas: [0, 1, 4, 2, 3, 5]
----------------------------------------
Paso 1:
  - Puntos de ruptura actuales: 3
  - Construyendo Hamiltoniano para 6 inversiones...
  - Inversión elegida: (1, 3)
  - Permutación actual: [0, 2, 1, 3]
----------------------------------------
Paso 2:
  - Puntos de ruptura actuales: 2
  - Construyendo Hamiltoniano para 6 inversiones...
  - Inversión elegida: (1, 2)
  - Permutación actual: [0, 1, 2, 3]
----------------------------------------
¡Permutación ordenada con éxito!
Permutación final: [0, 1, 2, 3]
Secuencia de inversiones (índices base 0): [(1, 3), (1, 2)]
Número total de inversiones: 2


Permutación inicial: [7, 6, 5, 4, 0, 3, 1, 2]
Permutación con centinelas: [0, 8, 7, 6, 5, 1, 4, 2, 3, 9]
----------------------------------------
Paso 1:
  - Puntos de ruptura actuales: 5
  - Construyendo Hamiltoniano para 28 inversiones...
  - Inversión elegida: (4, 5)
  - Permutación actual: [7, 6, 5, 4, 3, 0, 1,